# Import + drop des annonces dupliquées + drop des stages/contrats pro/alternances

In [1]:
import pandas as pd
import numpy as np
import re
import glob
from datetime import datetime, timedelta

df = pd.concat(map(pd.read_csv, glob.glob('Scraps/*.csv')))
df.to_csv('indeed_unclean.csv')

In [2]:
#options pandas
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = None

#import du csv
df = pd.read_csv('indeed_unclean.csv', encoding = 'utf8')
print(df.shape)

##Drop des colonnes inutiles
del df["Unnamed: 0"]
del df["Unnamed: 0.1"]

#drop des annonces sans description (annonces qui n'ont pas chargé complètement avant scrap)
df = df.dropna(subset=['description'])
df = df.dropna(subset=['description'])

#extraction des id d'annonces depuis les url + suppression des annonces doublons
# + nettoyage des strings qui ont un retour à la ligne
df = df.rename(columns={"id": "url"})
df['id'] = df['url'].str.extract('((?<=vjk=).{16})', expand=True)
df['id'] = df['id'].str.replace(r'\s*', '', regex=True)
df = df.drop_duplicates(subset='id', keep="first") # drop des annonces qui ont la même id
#df = df.drop_duplicates(subset='description', keep='first')  # drop des annonces qui ont la même description
df = df.sort_values('salaire').drop_duplicates('description', keep='first') # drop des annonces qui ont la même description
df = df.apply(lambda x: x.str.lower())
df = df.replace('\n','', regex=True)
df = df.replace('é','e', regex=True)
df = df.replace('ê','e', regex=True)
df = df.replace('è','e', regex=True)
df = df.replace('â','a', regex=True)
df = df.replace('à','a', regex=True)
df = df.replace('ù','u', regex=True)
df = df.replace('ç','c', regex=True)
df = df.replace('ô','o', regex=True)
df = df.replace('î','i', regex=True)

#1er retrait des annonces de stage et contrats pro/alternance en se basant sur les colonnes metas
df = df[~df['description'].str.contains('|'.join(['stage', 'alternance', 'gratification']))]
df['titre_poste'] = df['titre_poste'].replace(np.nan, '', regex=True)
df = df[~df['titre_poste'].str.contains('|'.join(['internship', 'stage', 'alternance', 'contrat pro',
                                 'professionnalisation', 'apprentissage', 'stagiaire']))]
df['contrat'] = df['contrat'].replace(np.nan, '', regex=True)
df = df[~df['contrat'].str.contains('|'.join(['alternance', 'professionnalisation', 'apprentissage',
                                                'stage', 'contrat pro']))]

#Reset de l'index + réorganisation des colonnes
df = df[['id', 'meta1', 'meta2', 'titre_poste', 'entreprise', 'localisation',
         'contrat', 'salaire', 'keyword_metier', 'keyword_localisation', 'date_publication', 'description', 'url']]

df.reset_index(drop=True, inplace=True)
print(df.shape)
print(df.info())

(36700, 14)
(16436, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16436 entries, 0 to 16435
Data columns (total 13 columns):
id                      16436 non-null object
meta1                   16251 non-null object
meta2                   16436 non-null object
titre_poste             16436 non-null object
entreprise              16381 non-null object
localisation            16403 non-null object
contrat                 16436 non-null object
salaire                 2237 non-null object
keyword_metier          16266 non-null object
keyword_localisation    16266 non-null object
date_publication        16436 non-null object
description             16436 non-null object
url                     16436 non-null object
dtypes: object(13)
memory usage: 1.6+ MB
None


# Salaire

In [3]:
#s = df[df['salaire'].isnull() == false]
#s = df[df['salaire'].isnull() == false]

df['salaire'] = df['salaire'].fillna('')

df['salary_type'] = ''
df['salary_clean'] = ''
df['salary'] = ''

df['salaire'] = df['salaire'].apply(lambda x: x.replace(" ", ""))  # delete all spaces
df['salaire'] = df['salaire'].apply(lambda x: x.replace(",", "."))  # cannot calculate the mean with , in the number

for i in range(df.shape[0]):
    if 'paran' in df['salaire'][i]:
        df['salary_type'][i] = 'y'
        df['salary_clean'][i] = df['salaire'][i][:-5]
        df['salary_clean'][i] = df['salary_clean'][i].replace('€', '')  # delete 'paran' et €, eg.50000 or 50000-60000
        if "-" in df['salary_clean'][i]:
            df['salary'][i] = df['salary_clean'][i].split("-")
            df['salary'][i] = (float(df['salary'][i][0]) + float(df['salary'][i][1])) / 2
        else:
            df['salary'][i] = float(df['salary_clean'][i])  # str to int

    elif 'parmois' in df['salaire'][i]:  # monthly salary has not been converted to annual salary
        df['salary_type'][i] = 'm'
        df['salary_clean'][i] = df['salaire'][i][:-7].replace('€', '')
        if "-" in df['salary_clean'][i]:
            df['salary'][i] = df['salary_clean'][i].split("-")
            df['salary'][i] = (float(df['salary'][i][0]) + float(df['salary'][i][1])) / 2
        else:
            df['salary'][i] = float(df['salary_clean'][i])

    elif 'parsemaine' in df['salaire'][i]:
        df['salary_type'][i] = 'w'
        df['salary_clean'][i] = df['salaire'][i][:-10].replace('€', '')
        if "-" in df['salary_clean'][i]:
            df['salary'][i] = df['salary_clean'][i].split("-")
            df['salary'][i] = (float(df['salary'][i][0]) + float(df['salary'][i][1])) / 2
        else:
            df['salary'][i] = float(df['salary_clean'][i])

    elif 'parjour' in df['salaire'][i]:
        df['salary_type'][i] = 'd'
        df['salary_clean'][i] = df['salaire'][i][:-7].replace('€', '')
        if "-" in df['salary_clean'][i]:
            df['salary'][i] = df['salary_clean'][i].split("-")
            df['salary'][i] = (float(df['salary'][i][0]) + float(df['salary'][i][1])) / 2
        else:
            df['salary'][i] = float(df['salary_clean'][i])

    elif 'parheure' in df['salaire'][i]:
        df['salary_type'][i] = 'h'
        df['salary_clean'][i] = df['salaire'][i][:-8].replace('€', '')
        if "-" in df['salary_clean'][i]:
            df['salary'][i] = df['salary_clean'][i].split("-")
            df['salary'][i] = (float(df['salary'][i][0]) + float(df['salary'][i][1])) / 2
        else:
            df['salary'][i] = float(df['salary_clean'][i])
    else:
        pass

df['salary_type'].value_counts()

     14199
y     1885
w      134
m      118
d       63
h       37
Name: salary_type, dtype: int64

# Date de publication

In [4]:
#cleaning initial 
df['date_publication'] = df['date_publication'].str.extract('((?<=il y a ).*)')
df['date_publication'] = df['date_publication'].str.replace('+','', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=minute).*)', 'minutes', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=minutes).*)', 'minutes', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=heure).*)', 'heures', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=heures).*)', 'heures', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=jour).*)', 'jours', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=jours).*)', 'jours', regex=True)
df['date_publication'] = df['date_publication'].str.replace('((?=mois).*)', 'mois', regex=True)

#remplacement des lignes avec le str heure/minute par "1 jours"
df['date_publication'] = df.date_publication.str.replace(r'.*heures.*', '1 jours')
df['date_publication'] = df.date_publication.str.replace(r'.*minutes.*', '1 jours')

#fonction de conversion des semaines/mois en jours
def convert_to_days(string):
    string = ''.join(string.split())
    digit = re.sub("\D", "", string) #extraction des chiffres du str
    if 'mois' in string: #conversion des mois en jours
        return str(int(digit) * 30)
    elif 'semaines' in string: #conversion des semaines en jours
        return str(int(digit) * 7)
    else:
        if len(digit) == 0:
            digit = '31'
        return digit
    return

#fonction d'ajustement de la date de l'annonce en fonction de la date du scrap
def adjust_date(annonce_date):
    scrap_date = datetime.strptime('2019-10-10', "%Y-%m-%d").date() #date du scrap
    new_date = scrap_date - timedelta(int(annonce_date))
    return new_date

#apply de la fonction sur l ensemble de la colonne
df['date_publication'] = df['date_publication'].apply(str)
df['date_publication'] = df['date_publication'].apply(lambda x: convert_to_days(x))
df['date_publication'] = df['date_publication'].apply(lambda x: adjust_date(x))

#conversion du type de la colonne en datetime
df['date_publication'] = pd.to_datetime(df['date_publication'], format='%Y-%m-%d')

# Type de contrat

In [5]:
#suppression des strings qui ne contiennent pas des informations sur le contrat
def check_contract_errors(string):  
    for char in list(string):
        if char.isdigit():
            return '' 
        
    if '€' in string:
        return ''
    elif '!' in string:
        return ''
    elif ':' in string:
        return ''    
    else:
        return string
    
df['contrat'] = df['contrat'].apply(lambda x: check_contract_errors(x))

df['contrat'] = df['contrat'].replace('', 'cdi') #remplacement des cases vides par cdi

#permet de déterminer le type de contrat par ligne
def set_contrat_type(string):  
    if 'cdi' in string:
        return 'cdi'
    elif 'cdd' in string:
        return 'cdd'
    elif 'freelance' in string:
        return 'freelance'   
    elif 'interim' in string:
        return 'interim'    
    elif 'temps plein' in string:
        return 'cdi'
    elif 'temps partiel' in string:
        return 'cdd'
    else:
        return string
    
df['contrat'] = df['contrat'].apply(lambda x: set_contrat_type(x))
print(df.shape)
df['contrat'].value_counts()

(16436, 16)


cdi          15298
freelance      848
cdd            202
interim         88
Name: contrat, dtype: int64

# Localisation

In [6]:
###Extract data from 'localisation'
df['localisation'] = df['localisation'].str.split(" ").str[-1].str.replace("(", "").str.replace(")", "")

#remplacement des nans par le keyword localisation
df['localisation'] = df['localisation'].fillna(df['keyword_localisation'])

# Use df['department'].value_counts() to check exceptions and solve these problems 
df['localisation'] = df['localisation'].replace("hauts-de-seine", "92")
df['localisation'] = df['localisation'].replace(["ile-de-france", "france", "paris"], "75")
df['localisation'] = df['localisation'].replace("rhone", "69")
df['localisation'] = df['localisation'].replace("occitanie", "31")  # occitanie is replaced by Toulouse Métropole
df['localisation'] = df['localisation'].replace("nouvelle-aquitaine",
                                      "33")  # nouvelle-aquitaine is replaced by Bordeaux Métropole
df['localisation'] = df['localisation'].replace("loire", "42")
df['localisation'] = df['localisation'].replace("haute-garonne", "31")
df['localisation'] = df['localisation'].replace("seine-saint-denis", "93")
df['localisation'] = df['localisation'].replace("essonne", "91")
df['localisation'] = df['localisation'].replace("charente-maritime", "17")
df['localisation'] = df['localisation'].replace("val-d'oise", "95")
df['localisation'] = df['localisation'].replace("vendee", "85")
df['localisation'] = df['localisation'].replace("loire-atlantique", "44")
df['localisation'] = df['localisation'].replace("val-de-marne", "94")
df['localisation'] = df['localisation'].replace("seine-et-marne", "77")
df['localisation'] = df['localisation'].replace("gers", "32")
df['localisation'] = df['localisation'].replace("auvergne-rhone-alpes", "69")  # auvergne-rhone-alpes is replaced by Grand Lyon
df['localisation'] = df['localisation'].replace("yvelines", "78")
df['localisation'] = df['localisation'].replace("gironde", "33")
df['localisation'] = df['localisation'].replace("ain", "01")
df['localisation'] = df['localisation'].replace("oise", "60")
df['localisation'] = df['localisation'].replace("maine-et-loire", "49")
df['localisation'] = df['localisation'].replace("eure-et-loir", "28")
df['localisation'] = df['localisation'].apply(lambda x: '75' if x in ('77', '78', '91', '92', '93', '94', '95') else x)  #grand Paris
df['localisation'] = df['localisation'].replace(["nantes", "35", "49", "85"], "44")
df['localisation'] = df['localisation'].replace(["lyon", "01", "38", "42", "26", "74", "73"], "69")
df['localisation'] = df['localisation'].replace(["27", "28", "60"], "75")
df['localisation'] = df['localisation'].replace(["81", "82", "47"], "33")

#on efface les villes qui ne figurent pas dans la liste suivante
cities = ['75', '69', '33', '44', '31']
df = df[df['localisation'].isin(cities)]

print(df['localisation'].value_counts())
print(df.info())
df.shape

75    9800
69    2652
44    1563
31    1537
33     822
Name: localisation, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16374 entries, 1 to 16435
Data columns (total 16 columns):
id                      16374 non-null object
meta1                   16191 non-null object
meta2                   16374 non-null object
titre_poste             16374 non-null object
entreprise              16319 non-null object
localisation            16374 non-null object
contrat                 16374 non-null object
salaire                 16374 non-null object
keyword_metier          16204 non-null object
keyword_localisation    16204 non-null object
date_publication        16374 non-null datetime64[ns]
description             16374 non-null object
url                     16374 non-null object
salary_type             16374 non-null object
salary_clean            16374 non-null object
salary                  16374 non-null object
dtypes: datetime64[ns](1), object(15)
memory usage: 2.1+ MB

(16374, 16)

# Titre du poste

In [7]:
# retrait h/f sur le titre des postes
df['titre_poste'] = df['titre_poste'].str.replace(r'\(?[f|h][\/|\-][f|h]\)?', '', regex=True) 
df['titre_poste'] = df['titre_poste'].str.replace(r'\(?[f|m][\/|\-][f|m]\)?', '', regex=True)
df['titre_poste'] = df['titre_poste'].str.replace("(e)", "")
df['titre_poste'] = df['titre_poste'].str.replace("-e", "")
df['titre_poste'] = df['titre_poste'].str.replace("\(h/f\)", "")
df['titre_poste'] = df['titre_poste'].str.replace("\(f/h\)", "")
df['titre_poste'] = df['titre_poste'].str.replace("h/f", "")
df['titre_poste'] = df['titre_poste'].str.replace("f/h", "")
df['titre_poste'] = df['titre_poste'].str.replace("-", "")
df['titre_poste'] = df['titre_poste'].str.replace("\/", "")
df['titre_poste'] = df['titre_poste'].str.replace("(", "")
df['titre_poste'] = df['titre_poste'].str.replace(")", "")

# Job Class

In [8]:
# Classification des jobs en data (= 1) ou dev (=0) selon le mot cle de recherche lors du scrap
data_job_list = ["data scientist", "data consultant", "performance analyst", "data engineer", "data miner",
                 "intelligence artificielle", "data manager", "data analyst"]

df['job_class'] = df['keyword_metier'].apply(lambda x: 1 if x in data_job_list else 0)
df['job_class'].value_counts()

0    10471
1     5903
Name: job_class, dtype: int64

# Extraction de salaire depuis la description

In [9]:
##duplication de la colonne description pour opérations regex sur salaire et skills
#df['temp1'] = df['description']
#df['temp1'] = df['temp1'].str.lower() #minuscule
#df['temp1'] = df['temp1'].str.replace(" ", "") #retrait des espaces
#df['temp1'] = df['temp1'].str.replace("é", "e")
#df['temp1'] = df['temp1'].str.replace("è", "e")
#df['temp1'] = df['temp1'].str.replace("à", "a")
#df['temp1'] = df['temp1'].str.replace("?", "")
#df['temp1'] = df['temp1'].str.replace("!", "")
#df['temp1'] = df['temp1'].str.replace("'", "")
#df['temp1'] = df['temp1'].str.replace("’", "")
#df['temp1'] = df['temp1'].str.replace("*", "")
#df['temp1'] = df['temp1'].str.replace("ç", "c")
#df['temp1'] = df['temp1'].str.replace("%", "")
#
#df['temp1'] = df['temp1'].str.replace("euros", "€")
#
#regex1 =r'(?<=\d.k)'
#df['temp1'] = df['temp1'].str.replace(regex1, '€', regex=True)
#df['temp1'] = df['temp1'].str.replace("€€", "€")
#
#df['temp1'] = df['temp1'].str.replace("mensuel", "/mois")
#df['temp1'] = df['temp1'].str.replace("parmois", "/mois")
#df['temp1'] = df['temp1'].str.replace("annuel", "/an")
#df['temp1'] = df['temp1'].str.replace("hebdomadaire", "/semaine")
#df['temp1'] = df['temp1'].str.replace("paran", "/an")
#df['temp1'] = df['temp1'].str.replace("parsemaine", "/semaine")
#df['temp1'] = df['temp1'].str.replace("parjour", "/jour")
#df['temp1'] = df['temp1'].str.replace("parheure", "/heure")
#df['temp1'] = df['temp1'].str.replace("horaire", "/heure")
#df['temp1'] = df['temp1'].str.replace("to", "a")
#df['temp1'] = df['temp1'].str.replace("month", "mois")
#df['temp1'] = df['temp1'].str.replace("year", "an")
#df['temp1'] = df['temp1'].str.replace("per", "par")
#df['temp1'] = df['temp1'].str.replace("et", "a")
#df['temp1'] = df['temp1'].str.replace("and", "a")
#df['temp1'] = df['temp1'].str.replace("//", "/")
#
#df['temp1'] = df['temp1'].str.replace("–", "-")
#df['temp1'] = df['temp1'].str.replace("-", "a")
#
##filtrage des lignes avec salaire null ET présence d indices contenant salaire dans la description
#pat = r'\d{1,6}[\,|\.]?\d{1,2}?k?[€|euros]?[à|a|to|\–|\-]?\d{1,6}[\,|\.]?\d{1,2}?k?€|euros?'
#pat2 = r'brut|parmois|\/mois|\/an|paran|\/jour|parjour|parheure|horaire|mensuel|€\/mois|\/heure|\/h|annuel|salaire:|remuneration:|gratification:|salary:|wage:?'
#pat3 = r'(€)'
#
#df['temp2'] = df['salaire'].isna() & df.description.str.contains(pat3)
#
#print(df['temp2'].value_counts())
#
#df['salaire_temp'] = df[df['temp2']==True]['temp1']
#
#regex ='((salaire:|remuneration:|gratification:|salary:|wage:)\S{1,64})'
#regex2= '(\/heure|\/jour|\/semaine|\/mois|\/an)'
#
#df['salaire_temp'] = df['salaire_temp'].str.extract(regex, expand=False)
#
#print(df['salaire_temp'].value_counts())
#
##cleaning salaire extrait de la description
#df['salaire_temp'] = df['salaire_temp'].str.replace("experience", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("profil", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("selon", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("le", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("niveau", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("detude", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("postuler", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("salaire", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("salary", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("brut", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace(":", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("et", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("(", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace(")", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("+", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("salaire:", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("remuneration:", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("gratification:", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("salary:", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("wage:", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("+", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace('^\D*$', '', regex=True) #tous les str sans chiffres
#
##df['salaire_temp'].count()
#
##Fillna pour éviter exceptions
#df['salaire_temp'] = df['salaire_temp'].fillna("")
#
##fonction permettant de multiplier les salaires avec 2 digits par 1000
#def salary_adjust(string1):
#    if string1 == "":
#        return string1
#    
#    if "-" in string1:
#        tmp = string1.split('-')  
#        
#        if (len(tmp[0]) == 3 or len(tmp[0]) == 4 ) and (len(tmp[1]) == 3 or len(tmp[1]) == 4):
#            tmp[0] = str(int(tmp[0]) * 12)
#            tmp[1] = str(int(tmp[1]) * 12)
#        elif (len(tmp[0]) == 2 and len(tmp[1])== 2):
#            tmp[0] = tmp[0] + "000"
#            tmp[1] = tmp[1] + "000"
#        else:
#            for i, item in enumerate(tmp):
#                if len(tmp[i]) == 2:
#                    tmp[i] = tmp[i] + "000"
#                elif len(tmp[i]) == 3:
#                    tmp[i] = tmp[i] + "00"
#                elif len(tmp[i]) == 4:            
#                    tmp[i] = str(int(tmp[i]) * 12)
#       
#                
#        result = tmp[0] + "-" + tmp[1]
#        return result    
#    else:
#        if len(string1) == 2:
#            return string1 + "000"
#        elif len(string1) == 3:            
#            return str(int(string1) * 12)
#        elif len(string1) == 4:            
#            return str(int(string1) * 12)
#        elif len(string1) >= 10:
#            return string1[:int(len(string1)/2)] + '-' + string1[int(len(string1)/2):]
#        elif len(string1) == 1:
#            return ""
#        else:
#            return string1
#
#
#df['salaire_temp'] = df['salaire_temp'].str.replace("33,000.00€a38,000.00€", "33000€a38000€")
#df['salaire_temp'] = df['salaire_temp'].str.replace("–", "-")
#df['salaire_temp'] = df['salaire_temp'].str.replace("-", "a")
#df['salaire_temp'] = df['salaire_temp'].str.replace(">", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("avan", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=annuel)\w*\D*', '', regex=True) 
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\/mois)\w*\D*', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\/an)\w*\D*', '', regex=True)  
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\/jour)\w*\D*', '', regex=True) 
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\/semaine)\w*\D*', '', regex=True) 
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\/heure)\w*\D*', '', regex=True) 
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'^\w{1-2}$', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'^0$', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\d.k)', '€', regex=True) #quand il n y a pas le signe €, on efface le str
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'^((?!€).)*$', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=\/mensuel)\\w*\D*', 'mois', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace("/an/an", "/an")
#df['salaire_temp'] = df['salaire_temp'].str.replace("remuneration", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("€€", "€")
#df['salaire_temp'] = df['salaire_temp'].str.replace("gratification", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("450000", "45000")
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=mois)(?s)(.*$)', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=an)(?s)(.*$)', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'(?<=an)(?s)(.*$)', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\,00€', '€', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\.00€', '€', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\,00', '€', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\.00', '€', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'0?\,000', '0000', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'0?\.000', '000', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\/mois?.*', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\/an?.*', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace("/", "a")
#df['salaire_temp'] = df['salaire_temp'].str.replace("k", "000")
#
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'\/€?.*', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'[^€]*$', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'[^\d]*$', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace("a", "-")
#df['salaire_temp'] = df['salaire_temp'].str.replace(r'^(\D*)', '', regex=True)
#df['salaire_temp'] = df['salaire_temp'].str.replace("€", "")
#
#df['salaire_temp'] = df['salaire_temp'].apply(lambda x: salary_adjust(x))
#df['salaire_temp'] = df['salaire_temp'].str.replace(">", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace(" ", "")
#df['salaire_temp'] = df['salaire_temp'].str.replace("-", "€ - ")
#df['salaire_temp'] = df['salaire_temp'].astype(str) + '€ par an'
#
#df['salaire_temp'].replace("", np.nan, inplace=True) 
#
#print(df['salaire'].count())
#
##df['salaire'] = df['salaire'].fillna(df['salaire_temp'])
#
#print(df['salaire'].count())
#print(df['salaire'].value_counts())
#
## suppression des colonnes temporaires
#del df['temp1']
#del df['temp2']
#del df['salaire_temp']
#
#df.head()
##df['salaire'] = df['salaire'].fillna("")

# Experience

In [10]:
#duplication de la colonne description pour opérations regex sur salaire et skills
df['temp1'] = df['description']
df['temp1'] = df['temp1'].str.lower() #minuscule
df['temp1'] = df['temp1'].str.replace(" ", "") #retrait des espaces
df['temp1'] = df['temp1'].str.replace("é", "e")
df['temp1'] = df['temp1'].str.replace("è", "e")
df['temp1'] = df['temp1'].str.replace("à", "a")
df['temp1'] = df['temp1'].str.replace("?", "")
df['temp1'] = df['temp1'].str.replace("!", "")
df['temp1'] = df['temp1'].str.replace("'", "")
df['temp1'] = df['temp1'].str.replace("’", "")
df['temp1'] = df['temp1'].str.replace("*", "")
df['temp1'] = df['temp1'].str.replace("ç", "c")
df['temp1'] = df['temp1'].str.replace("%", "")
df['temp1'] = df['temp1'].str.replace(":", "")
df['temp1'] = df['temp1'].str.replace("/", "")
df['temp1'] = df['temp1'].str.replace("'", "")
df['temp1'] = df['temp1'].str.replace("(", "")
df['temp1'] = df['temp1'].str.replace(")", "")
df['temp1'] = df['temp1'].str.replace(",", "")
df['temp1'] = df['temp1'].str.replace(".", "")
df['temp1'] = df['temp1'].str.replace(">", "")
df['temp1'] = df['temp1'].str.replace("+", "")

df['temp1'] = df['temp1'].str.replace("significative", "")
df['temp1'] = df['temp1'].str.replace("daumoins", "")
df['temp1'] = df['temp1'].str.replace("min.requis", "")
df['temp1'] = df['temp1'].str.replace("requis", "")
df['temp1'] = df['temp1'].str.replace("minimum", "")
df['temp1'] = df['temp1'].str.replace("reussie", "")
df['temp1'] = df['temp1'].str.replace("deplus", "")
df['temp1'] = df['temp1'].str.replace("similaire", "")
df['temp1'] = df['temp1'].str.replace("ou", "")
df['temp1'] = df['temp1'].str.replace("de", "")
df['temp1'] = df['temp1'].str.replace("dune", "")
df['temp1'] = df['temp1'].str.replace("ustifiez", "")
df['temp1'] = df['temp1'].str.replace("justifiez", "")
df['temp1'] = df['temp1'].str.replace("beneficiez", "")
df['temp1'] = df['temp1'].str.replace("master", "")
df['temp1'] = df['temp1'].str.replace("dau", "")
df['temp1'] = df['temp1'].str.replace("souhaitee", "")
df['temp1'] = df['temp1'].str.replace("souhaite", "")
df['temp1'] = df['temp1'].str.replace("professionnelle", "")

df['temp1'] = df['temp1'].str.replace("an", "ans")
df['temp1'] = df['temp1'].str.replace("annee", "ans")
df['temp1'] = df['temp1'].str.replace("annees", "ans")
df['temp1'] = df['temp1'].str.replace("annes", "ans")
df['temp1'] = df['temp1'].str.replace("years", "ans")
df['temp1'] = df['temp1'].str.replace("year", "ans")
df['temp1'] = df['temp1'].str.replace("1an", "1ans")

#filtrage des lignes avec salaire null ET présence d indices contenant salaire dans la description
df['temp2'] = (df['salaire'].notna() | df['salaire'].notnull())
#print(df.temp2.value_counts())

#fonction permettant d'ajuster les années d'exp selon le str
#def exp_adjust(string1):
#    if int(string1) > 15:
#        return ""
#    else:
#        return string1

df['experience'] = df[df['temp2']==True]['temp1']

regex ='(\S{1,16}experience\S{1,128})'
df['experience'] = df['experience'].str.extract(regex, expand=False)

df['experience'] = df['experience'].str.replace(r'\d*€', '', regex=True) #salaire proche de l'exp
df['experience'] = df['experience'].str.replace(r'bac\+\d{1}\/?d?', '', regex=True)  
df['experience'] = df['experience'].str.replace("-", "a")
df['experience'] = df['experience'].str.replace("et", "a")
df['experience'] = df['experience'].str.replace("#", "")
df['experience'] = df['experience'].str.replace(";", "")
df['experience'] = df['experience'].str.replace("–", "")

df['experience'] = df['experience'].str.extract('(\S{1,10}ans\S{1,16})', expand=True)
df['experience'] = df['experience'].str.replace('\d*k', '', regex=True) #tous les chffres se terminant par k (des salaires)
df['experience'] = df['experience'].str.extract('((.*?)ans)', expand=True)
df['experience'] = df['experience'].str.replace("xperience", "")
df['experience'] = df['experience'].str.replace("a", "-")
df['experience'] = df['experience'].str.replace("-ns", "")
df['experience'] = df['experience'].str.replace('([a-z]+)', '', regex=True) #toutes les lettres
df['experience'] = df['experience'].str.replace("--", "")
df['experience'] = df['experience'].str.replace("---", "")
df['experience'] = df['experience'].str.replace('^\-', '', regex=True)
df['experience'] = df['experience'].str.replace('\-$', '', regex=True)
df['experience'] = df['experience'].str.replace("23", "2-3")
df['experience'] = df['experience'].str.replace("24", "2-4")
df['experience'] = df['experience'].str.replace("25", "2-5")
df['experience'] = df['experience'].str.replace("2-2-5", "2-5")
df['experience'] = df['experience'].str.replace("15", "1-5")
df['experience'] = df['experience'].str.replace("35", "3-5")
df['experience'] = df['experience'].str.replace("5-5-7", "5-7")
df['experience'] = df['experience'].str.replace("45", "4-5")
df['experience'] = df['experience'].str.replace("54", "4-5")
df['experience'] = df['experience'].str.replace("4-5-5", "4-5")
df['experience'] = df['experience'].str.replace("231", "2-3")
df['experience'] = df['experience'].str.replace("5-2", "5")
df['experience'] = df['experience'].str.replace("2-31", "2-3")
df['experience'] = df['experience'].str.replace("10-1-5", "1-5")
df['experience'] = df['experience'].str.replace("-3", "3")
df['experience'] = df['experience'].str.replace("63", "3")
df['experience'] = df['experience'].str.replace("03", "3")
df['experience'] = df['experience'].str.replace("13", "1-3")
df['experience'] = df['experience'].str.replace('^\D*$', '', regex=True) #tous les str sans chiffre
df['experience'] = df['experience'].str.replace("23", "2-3")
df['experience'] = df['experience'].str.replace("23", "2-3")
df['experience'] = df['experience'].str.replace("~", "2-3")
df['experience'] = df['experience'].str.replace("&-5", "2-5")
df['experience'] = df['experience'].str.replace("2-3-510", "3-5")
df['experience'] = df['experience'].str.replace("4-5968", "4-5")
df['experience'] = df['experience'].str.replace("98", "")
df['experience'] = df['experience'].str.replace("75", "")
df['experience'] = df['experience'].str.replace("00", "")
df['experience'] = df['experience'].str.replace("26", "2-6")
df['experience'] = df['experience'].str.replace("360", "3-6")
df['experience'] = df['experience'].str.replace("310", "3")
df['experience'] = df['experience'].str.replace("50", "5")
df['experience'] = df['experience'].str.replace("37", "3")
df['experience'] = df['experience'].str.replace("4-6-10", "4-6")
df['experience'] = df['experience'].str.replace("2•", "2")
df['experience'] = df['experience'].str.replace("2•", "")
df['experience'] = df['experience'].str.replace("127-10", "")
df['experience'] = df['experience'].str.replace("68", "6-8")
df['experience'] = df['experience'].str.replace("51", "5")
df['experience'] = df['experience'].str.replace("量。", "")
df['experience'] = df['experience'].str.replace("<", "")
df['experience'] = df['experience'].str.replace("33", "3")
df['experience'] = df['experience'].str.replace("¤", "")
df['experience'] = df['experience'].str.replace("23", "3")
df['experience'] = df['experience'].str.replace("09911", "")
df['experience'] = df['experience'].str.replace("”", "")
df['experience'] = df['experience'].str.replace("·", "")
df['experience'] = df['experience'].str.replace("12-3-4", "3-4")
df['experience'] = df['experience'].str.replace("\[]", "")
df['experience'] = df['experience'].str.replace("53", "3")
df['experience'] = df['experience'].str.replace("48", "4-8")
df['experience'] = df['experience'].str.replace("", "")
df['experience'] = df['experience'].str.replace("4160", "")
df['experience'] = df['experience'].str.replace("°", "")
df['experience'] = df['experience'].str.replace("27", "")
df['experience'] = df['experience'].str.replace("50", "5")
df['experience'] = df['experience'].str.replace("101", "1")
df['experience'] = df['experience'].str.replace("46", "4-6")
df['experience'] = df['experience'].str.replace("56", "5-6")
df['experience'] = df['experience'].str.replace("22", "2")
df['experience'] = df['experience'].str.replace("17", "1-7")
df['experience'] = df['experience'].str.replace("60", "")
df['experience'] = df['experience'].str.replace("121", "1-2")
df['experience'] = df['experience'].str.replace("0&", "")
df['experience'] = df['experience'].str.replace("810", "8-10")
df['experience'] = df['experience'].str.replace("612", "6-12")
#df['experience'] = df['experience'].apply(lambda x: exp_adjust(x))

df['experience'].replace('', np.nan, inplace=True) 
print(df['experience'].value_counts())

2        767
5        757
3        729
1        175
4        170
        ... 
3-9        1
2-8        1
10-12      1
18         1
04         1
Name: experience, Length: 68, dtype: int64


# Level

In [11]:
# transformation de l'exp en 3 catégories

df['level'] = df['experience']
df['level'] = df['level'].replace(np.nan, '', regex=True)

# permet d'attribuer le skill en fonction de l'exp
# category 1: <3 years
# category 2: 3-5 years
# category 3: >5 years
# category 4: == 0 -> level not set

def set_level(string):
    if "-" in string:
        tmp = string.split('-')            
        if (int(tmp[0]) > int(tmp[1])):
            tmp = int(tmp[0])
        else:
            tmp = int(tmp[1])
    else:
        if(string == ""):
            return "0"
        else:
            tmp = int(string)
    
    if tmp < 3:
        return "1"
    elif tmp > 5:
        return "3"
    else:
        return "2"

df['level'].value_counts()

# si l'exp n est pas spécifiée on la fixe à 1 par défaut

#filtrage des lignes avec salaire null ET experience non null
df['temp3'] = (df['experience'].notna() | df['experience'].notnull())

print(df['level'].value_counts())

df['level'] = df['level'].apply(lambda x: set_level(x))

df['level'] = df['level'].astype(int)
df['level'] = df['level'].replace(0, np.nan)

# suppression des colonnes temporaires
del df['temp1']
del df['temp2']
del df['temp3']

df['level'].value_counts()

        12798
2         767
5         757
3         729
1         175
        ...  
9           1
1-4         1
4-10        1
7-12        1
3-8         1
Name: level, Length: 69, dtype: int64


2.0    2085
1.0     988
3.0     503
Name: level, dtype: int64

# Reset index

In [12]:
df['salaire'] = df['salaire'].replace('', np.nan)
#df['salaire'].value_counts().reset_index().to_csv('df.csv')

df.reset_index(drop=True, inplace=True)

# Modèle pour fillna les salaires sans experiences

In [13]:
def remplissage_salaire(df):
    nouveau_salaire= []
    for idx, elt in enumerate(df.salary_clean):
        if '-' in str(elt):
            elt=list(map(float, df.salary_clean[idx].split("-")))
            a=abs(elt[0]-elt[1])/max(df.level)
            #y= ax+b une regression lineaire pour les salaires en fonction des skills
            b=min(elt)
            y=a*df.level[idx] + b
            nouveau_salaire.insert (idx, y)
        else:    
            nouveau_salaire.insert (idx, elt)
    df['nouveau_salaire'] = nouveau_salaire
    return df 

def separation_salaire(df):
    min_salaire=[]
    max_salaire=[]
    moyenne_salaire=[]
    for idx, elt in enumerate(df.salary_clean):   
        if '-' in str(elt):
            a=list(map(float, df.salary_clean[idx].split("-")))
            m=min(a)
            M=max(a)
                
            max_salaire.insert(idx, M)
            min_salaire.insert(idx, m)
            moyenne_salaire.insert(idx, M/2+m/2)
        else:
            max_salaire.insert(idx, float(elt))
            min_salaire.insert(idx, float(elt)) 
            moyenne_salaire.insert(idx, float(elt))
    df['min_salaire']=min_salaire
    df['max_salaire']=max_salaire
    df['moyenne_salaire']= moyenne_salaire
    return df

df = remplissage_salaire(df)

#df.salary_clean.unique()
df.salary_clean = df['salary_clean'].replace("", np.nan)
df = separation_salaire(df)

# Export du dataset

In [14]:
print(df.shape)
print(df.info())

df.to_csv('./indeed_cleaned.csv')

(16374, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16374 entries, 0 to 16373
Data columns (total 23 columns):
id                      16374 non-null object
meta1                   16191 non-null object
meta2                   16374 non-null object
titre_poste             16374 non-null object
entreprise              16319 non-null object
localisation            16374 non-null object
contrat                 16374 non-null object
salaire                 2229 non-null object
keyword_metier          16204 non-null object
keyword_localisation    16204 non-null object
date_publication        16374 non-null datetime64[ns]
description             16374 non-null object
url                     16374 non-null object
salary_type             16374 non-null object
salary_clean            2229 non-null object
salary                  16374 non-null object
job_class               16374 non-null int64
experience              3576 non-null object
level                   3576 non-null float64
n

# MongoDB

In [ ]:
from pymongo import MongoClient

client=MongoClient()
db = client.Database  #Nom database
table = db.Col  #Nom collection
table
table.insert_many(df.to_dict(orient='record'))